[Reference](https://levelup.gitconnected.com/dumbell-plots-slope-charts-and-mirror-bar-charts-in-python-and-plotly-55dd0e1478a5)

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# 1. Dumbell Plots

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/hgarg01/graphs-for-comparing-values/main/data.csv')
data.shape

(16800, 17)

In [3]:
data.columns

Index(['Country', 'ISO', 'Sex', 'Year', 'Mean_BMI_adults',
       'Prevalence_obesity_adults', 'Prevalence_underweight_adults',
       'Prevalence_morbid_obesity_adults', 'Diabetes_prevalence',
       'Systolic_blood_pressure', 'Prevalence_raised_blood_pressure', 'Region',
       'Superregion', 'Years_of_education', 'Urbanisation',
       'Western_diet_score', 'GDP_USD'],
      dtype='object')

In [4]:
df_africa = data[(data['Superregion']=='Sub-Saharan Africa') & (data['Sex'] == 'Male')& ((data['Year'] == 2015) | (data['Year'] == 1975))]
df_africa_gdp = df_africa[['Country', 'Year', 'GDP_USD']]

In [5]:
df_africa_gdp

,Country,Year,GDP_USD
421,Angola,1975,4646.600586
501,Angola,2015,7626.981445
1597,Benin,1975,1242.958740
1677,Benin,2015,1800.357910
2017,Botswana,1975,2710.295654
...,...,...,...
15789,Uganda,2015,1432.712158
16633,Zambia,1975,3548.811035
16713,Zambia,2015,3257.269287
16717,Zimbabwe,1975,2592.614258


In [6]:
df_africa_gdp['Year'] = df_africa_gdp['Year'].astype(str) 
#plot the points
fig = px.scatter(df_africa_gdp, x="GDP_USD", y="Country", color = 'Year', color_discrete_map= {'2015': '#0e668b', '1975': '#a3c4dc' }, hover_name = 'Country' )
fig.update(layout_showlegend=False)

# iterate on each country
for i in df_africa_gdp["Country"].unique():
    # filter by country
    df_country = df_africa_gdp[df_africa_gdp["Country"] == i]
    
    fig.add_shape(
        type="line", opacity = 0.8,
        layer="below",
        # connect the two markers
        y0=df_country.Country.values[0], x0=df_country.GDP_USD.values[0],
        y1=df_country.Country.values[1], x1=df_country.GDP_USD.values[1], 
        line=dict(color="black",width=1)
    )
    fig.update_traces(marker=dict(size=10, opacity = 0.9))
fig.update_layout(
     xaxis=dict(showgrid=True), 
     yaxis=dict(showgrid=False),
     title = "Comparison of GDP of Sub-Saharan African countries in the years 1975 and 2015"
)

fig.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# 2. Slope Charts

In [7]:
#slice the data
ocenia = data[(data['Superregion'] == 'Oceania') & (data['Sex'] == 'Male') & ((data['Year'] == 1975) | (data['Year']== 2015))]
#select the columns we are interested in
ocenia_gdp =ocenia[['Country', 'Year', 'GDP_USD']]
#pivot the data
ocenia_gdp1 = pd.pivot_table(ocenia_gdp, index = 'Country', columns = 'Year', values = 'GDP_USD' ).reset_index()
#calculate change in GDP across teh 2 years
ocenia_gdp1['Change in GDP'] = ocenia_gdp1[2015] - ocenia_gdp1[1975]

#convert the change in gdp to direction - up and down
bins = [ocenia_gdp1['Change in GDP'].min()-1, 0,ocenia_gdp1['Change in GDP'].max() +1]
labels = ['down', 'up']
ocenia_gdp1['direction'] = pd.cut(ocenia_gdp1['Change in GDP'], bins=bins, labels=labels)
#replace the values up and down with colors to be used in the plot
ocenia_gdp1['direction color'] = ocenia_gdp1['direction'].replace('up', '#1a9d41').replace('down', '#f15656')

ocenia_gdp1

Year,Country,1975,2015,Change in GDP,direction,direction color
0,American Samoa,8588.920981,9445.126868,856.205887,up,#1a9d41
1,Cook Islands,2635.915771,24761.236330,22125.320559,up,#1a9d41
2,Fiji,5459.113770,7593.343750,2134.229980,up,#1a9d41
3,French Polynesia,18857.996830,34683.409580,15825.412750,up,#1a9d41
4,Kiribati,4563.722656,1784.164429,-2779.558227,down,#f15656
5,Marshall Islands,2341.586670,3777.478516,1435.891846,up,#1a9d41
6,Micronesia (Federated States of),3183.541748,3556.715576,373.173828,up,#1a9d41
7,Nauru,19849.666020,6132.740234,-13716.925786,down,#f15656
8,Niue,5555.319064,25005.998050,19450.678986,up,#1a9d41
9,Palau,38993.152340,15364.714840,-23628.437500,down,#f15656


In [8]:
#create scatter plots for the values in the 2 years
fig = go.Figure(go.Scatter(x = [0] * len(ocenia_gdp1), y =ocenia_gdp1[1975] ,  mode = 'lines+markers+text', showlegend = False, hovertext = ocenia_gdp1['Country'], name = 'GDP in 1975',marker=dict(
            color='#3970e7', 
            size=10)
))
fig.add_trace(go.Scatter( x = [1] * len(ocenia_gdp1), y = ocenia_gdp1[2015], mode = 'lines+markers+text', showlegend = False, hovertext = ocenia_gdp1['Country'], name = 'GDP in 2015',marker=dict(
            color='#3970e7',
            size=10)
))

#add the lines
for y0, y1, c in zip(ocenia_gdp1[1975], ocenia_gdp1[2015], ocenia_gdp1['direction color']):
    fig.add_shape(type='line', x0=0, x1=1, y0=y0, y1=y1, line=dict(
        color=c))
#Add text on top of the lines
fig.add_annotation(x=0, y=ocenia_gdp1[1975].max()+500,
            text="<b>GDP in 1975</b>",
            showarrow=False,
            yshift=10, font = dict(color = 'black'))
fig.add_annotation(x=1, y=ocenia_gdp1[1975].max()+500,
            text="<b>GDP in 2015</b>",
            showarrow=False,
            yshift=10, font = dict(color = 'black'))

#Add the names of the country
for i in range(len(ocenia_gdp1)):
    if i%2 == 1:
        fig.add_annotation(x = 0-0.04, y = ocenia_gdp1.iloc[i][1975], xanchor = 'right', text = ocenia_gdp1.iloc[i]['Country'], showarrow = False)
    else:
        fig.add_annotation(x = 1+0.04, y = ocenia_gdp1.iloc[i][2015], xanchor = 'left', text = ocenia_gdp1.iloc[i]['Country'], showarrow = False)

#adjust the axes and titles
fig.update_layout(title = "<b>Comparison of GDP of countries in Ocenia region in the years 1975 and 2015</b>", height = 800)
fig.update_xaxes(showticklabels = False)
fig.show()

# 3. Mirror Bar Chart

In [9]:
df_africa_gdp

,Country,Year,GDP_USD
421,Angola,1975,4646.600586
501,Angola,2015,7626.981445
1597,Benin,1975,1242.958740
1677,Benin,2015,1800.357910
2017,Botswana,1975,2710.295654
...,...,...,...
15789,Uganda,2015,1432.712158
16633,Zambia,1975,3548.811035
16713,Zambia,2015,3257.269287
16717,Zimbabwe,1975,2592.614258


In [10]:
# # create subplots
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots
# fig = make_subplots(rows=1, cols=2, specs=[[{}, {}]], shared_xaxes=True,
#                     shared_yaxes=True, horizontal_spacing=0, subplot_titles=['<-- GDP of countries in 1975', 'GDP of countries in 2015 -->'])
# #add bars to teh figure
# fig.append_trace(go.Bar(name = 'GDP in 1975', y=df_africa_gdp1['Country'], x=df_africa_gdp1[1975], orientation='h', width=0.4, showlegend=False, marker_color='#80bff3', text = np.round(np.abs(df_africa_gdp1['1975 in negative']), -3)), 1, 1)
# fig.append_trace(go.Bar(name = 'GDP in 2015', y=df_africa_gdp1['Country'], x=df_africa_gdp1[2015], orientation='h', width=0.4, showlegend=False, marker_color='#0e668b', text = np.round(df_africa_gdp1[2015], -3)), 1, 2)

# #specify the hover template
# fig.update_traces(
#     hovertemplate="<br>".join([
#         "<b>%{y}</b>",
#         "GDP : %{x}"]))
# fig.update_traces(textposition='outside')

# fig.update_layout(title = "<b>Comparison of GDP of countries in Sub-Saharan African region in the years 1975 and 2015</b>", height = 800)
# fig.update_layout(height = 1000)

# #reverse the range so that the left plor gets plotted from right to left
# fig['layout']['xaxis1']['autorange'] = "reversed"
# fig.show()